In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # on NERSC filelocking is not allowed
import h5py
import tensorflow as tf
# Make notebook run on other GPUS. GPT's solution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(gpus[2], 'GPU')  # change 1 to 0, 2, 3 as needed
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# import tensorflow.math as tfmath
import tensorflow.keras as keras
# from scipy.optimize imporjun26t curve_fit
# from tensorflow.keras import layers, Model
# import tensorflow.keras.backend as K
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.models import load_model
# from sklearn.metrics import roc_curve, auc
# import sklearn.metrics as sk
# from tensorflow.keras.models import Model
from tensorflow.keras.layers import PReLU, Input, LSTM, Flatten, Concatenate, Dense, Conv2D, TimeDistributed, MaxPooling2D, LeakyReLU, ReLU, Dropout, BatchNormalization, Activation
# from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.metrics import Precision
# # from qkeras import QActivation, QDense, QConv2D, QBatchNormalization, QConv2DBatchnorm # These don't seem to be used
# # from qkeras import quantized_relu, quantized_bits
# from tensorflow.keras.regularizers import l1, l2, l1_l2

In [2]:
# Load data
home_path = "/global/cfs/cdirs/m2616/jananinf/projsIO/ToyVAE/" # Updated to NERSC
file_path = home_path + "preprocessed_SNL_data.h5"
with h5py.File(file_path, 'r') as hf:           # Shapes:
    X_train = hf['X_train'][:]                  # (3200000, 57)
    X_test  = hf['X_test'][:]                   # (800000,  57)
    Ato4l_data  = hf['Ato4l_data'][:]           # (55969,   57) Signal data? 
    hToTauTau_data  = hf['hToTauTau_data'][:]   # (691283,  57)
    hChToTauNu_data  = hf['hChToTauNu_data'][:] # (760272,  57)
    leptoquark_data = hf['leptoquark_data'][:]  # (340544,  57)
    print("Data loaded from preprocessed_SNL_data.h5")

Data loaded from preprocessed_SNL_data.h5


In [ ]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    


In [ ]:
def Qmake_encoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    """
    Makes encoder

    Parameters
    ----------
    input_dim : int
        size of input layer
    h_dim_[X] : int
        size of hidden layer X
    latent_dim : int
        size of latent layer
    """

    # What is this and why? ----------------------------------------------------
    # l2_factor = 1e-3 
    # --    

    # Input layer --------------------------------------------------------------
    inputs = keras.Input(shape=(input_dim))
    x = BatchNormalization(name="BN_Input")(inputs) # paper indicates batch 
                                               # normalization at the beginning
                                               # of each layer
    # --

    # Hidden layer 1 -----------------------------------------------------------
    x = Dense(h_dim_1,
    # I am not quite seeing these in the paper. Maybe they're implied?
            #  kernel_initializer=keras.initializers.HeNormal(seed=None), 
            #  bias_initializer=keras.initializers.Zeros(),
            #  kernel_regularizer=l1_l2(l1=0, l2=l2_factor), # This is where the l2_factor is used.

             name = "dense1")(inputs)
    x = BatchNormalization(name="BN_h1")(x)
    x = LeakyReLU(name="L_relu1")(x)
    # ---

    # Hidden Layer 1 -----------------------------------------------------------
    x = Dense(h_dim_2,
            #  kernel_initializer=keras.initializers.HeNormal(seed=None),
            #  bias_initializer=keras.initializers.Zeros(),
            #  kernel_regularizer=l1_l2(l1=0, l2=l2_factor),
             name = "dense2")(x)    
    x = BatchNormalization(name="BN_h2")(x)
    x = LeakyReLU(name="L_relu2")(x)
    # ---

    # Latent layer -------------------------------------------------------------
    # No activation. 
    z_mean=Dense(latent_dim, name='z_mean',
                #   kernel_initializer=keras.initializers.HeNormal(seed=None),
                #   bias_initializer=keras.initializers.Zeros(),
                #   kernel_regularizer=l1_l2(l1=0, l2=l2_factor)
                )(x)
    z_logvar=Dense(latent_dim, name='z_log_var',
                    #   kernel_initializer=keras.initializers.Zeros(),
                    #   bias_initializer=keras.initializers.Zeros(),
                    #   kernel_regularizer=l1_l2(l1=0, l2=l2_factor)
                    )(x)
    z=Sampling()([z_mean,z_logvar])
    # ---


    encoder = keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder


def Qmake_decoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    """
    Makes decoder

    Parameters
    ----------
    input_dim : int
        size of input layer
    h_dim_[X] : int
        size of hidden layer X
    latent_dim : int
        size of latent layer
    """
    # l2_factor = 1e-3
    inputs=keras.Input(shape=(latent_dim)) # Old
    # inputs=keras.Input(shape=(latent_dim,)) # GPT Fix: Int vs tuple
    x=layers.Dense(h_dim_2,
                #    activation='relu',
                #    kernel_initializer=keras.initializers.HeNormal(seed=None),
                #    bias_initializer=keras.initializers.Zeros(),
                #    kernel_regularizer=l1_l2(l1=0, l2=l2_factor)
                   )(inputs)
    x=layers.Dense(h_dim_1,
                   activation='relu',
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros(),
                   kernel_regularizer=l1_l2(l1=0, l2=l2_factor))(x)
    z=layers.Dense(input_dim,
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros(),
                   kernel_regularizer=l1_l2(l1=0, l2=l2_factor))(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

In [11]:
INPUT_SZ = 57
H1_SZ = 32
H2_SZ = 16
LATENT_SZ = 3
enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
enc.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 57)]         0           []                               
                                                                                                  
 dense1 (Dense)                 (None, 32)           1856        ['input_2[0][0]']                
                                                                                                  
 BN_h1 (BatchNormalization)     (None, 32)           128         ['dense1[0][0]']                 
                                                                                                  
 relu1 (ReLU)                   (None, 32)           0           ['BN_h1[0][0]']                  
                                                                                            